## Using modules with VPRTempo and VPRTempoQuant

### By Adam D Hines (https://research.qut.edu.au/qcr/people/adam-hines/)

VPRTempo is based on the following paper, if you use or find this code helpful for your research please consider citing the source:
    
[Adam D Hines, Peter G Stratton, Michael Milford, & Tobias Fischer. "VPRTempo: A Fast Temporally Encoded Spiking Neural Network for Visual Place Recognition. arXiv September 2023](https://arxiv.org/abs/2309.10225)

### Introduction

In this tutorial, we will go through how to use modules with VPRTempo. Modules break up the training data into multiple networks, which has been shown to [improve the overall performance](https://towardsdatascience.com/machine-learning-with-expert-models-a-primer-6c74585f223f) and accuracy of larger models.

Before starting, make sure you have [installed the dependencies](https://github.com/AdamDHines/VPRTempo-quant#installation-and-setup) and/or activated the conda environment. You will also need the [Nordland](https://github.com/AdamDHines/VPRTempo-quant#nordland) dataset before proceeding, as this tutorial will cover training the network using this as an example.

### Comparing results using expert modules for VPRTempo

Let's start by training the base model with 1000 places, which is 500 more than the default settings. We will need to parse the `--train_new_model`, `--num_places`, as well as another argument we haven't seen yet `--max_module`. 

`--max_module` tells the network how many places each expert module should learn, which by default is set to `500`. So if we're training a new, singular network with 1000 places we need to increase `max_module` to 1000.

In [ ]:
# Change the working directory to the main folder from tutorials
import os
os.chdir('../')

In [ ]:
# Train a single network with 1000 places
!python main.py --num_places 1000 --max_module 1000 --train_new_model

Now let's see how this performs.

In [ ]:
# Run the inferencing network on the singular 1000 place trained model
!python main.py --num_places 1000 --max_module 1000

Performance here is still pretty good, but let's see if we can improve it by splitting up the network into modules!

Now that splitting up our 1000 place network into 2 networks, we can remove the `--max_module` argument because the default is set to 500. Instead what we will parse is `--num_modules` to tell the network to split things up into two models.

In [ ]:
# Train a new 1000 place model with 2 modules
!python main.py --num_places 1000 --num_modules 2 --train_new_model

Now let's see how it compares with the singular model.

In [ ]:
# Run the network with 2 modules
!python main.py --num_places 1000 --num_modules 2

A modest boost to performance, however you have to imagine how this scales to much larger networks - especially when considering training times. Because the output layer is one-hot encoded, you need to increase the number of output neurons with each place you want to learn. Splitting up networks has a key benefit for VPRTempo to reduce overall training times with little impact on inference speeds. 

(Optional) Run a single network for 2500 places or 5 expert modules for 500 places each (reduced dimensionality to speed things up).

In [ ]:
# Optional: run a 2500 place comparison for singular vs modular networks
# Train networks
!python main.py --num_places 2500 --max_module 2500 --dims 28,28 --patches 7 --train_new_model
!python main.py --num_places 2500 --num_modules 5 --dims 28,28 --patches 7 --train_new_model
# Run inference
!python main.py --num_places 2500 --max_module 2500 --dims 28,28 --patches 7
!python main.py --num_places 2500 --num_modules 5 --dims 28,28 --patches 7

As in the other tutorials, parsing the `--quantize` argument will run exactly the same but for VPRTempoQuant. Let's do a quick comparison.

In [ ]:
# Train networks
#!python main.py --num_places 1500 --max_module 1500 --dims 28,28 --patches 7 --train_new_model --quantize
#!python main.py --num_places 1500 --num_modules 3 --dims 28,28 --patches 7 --train_new_model --quantize
# Run inference
!python main.py --num_places 1500 --max_module 1500 --dims 28,28 --patches 7 --quantize
!python main.py --num_places 1500 --num_modules 3 --dims 28,28 --patches 7 --quantize

Once again, we can see that whilst there's a modest boost to the accuracy result the clear improve is the training speed. Because each network is smaller, the opeations on CPU are a lot less computationally heavy when splitting the networks up.

### Conclusions

This tutorial provided a simple overview of how you can train network models using expert modules. 

If you would like to go more in-depth, checkout some of the [other tutorials](https://github.com/AdamDHines/VPRTempo-quant/tree/main/tutorials).